In [1]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("./combined.csv")
df.head()

/Users/tegochang/opt/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (28,29) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,region,month
0,0,108061,https://www.airbnb.com/rooms/108061,20211215062309,2021-12-15,Walk to stores/parks/downtown. Fenced yard/Pet...,Walk to town in ten minutes! Monthly rental in...,"I love my neighborhood! Its friendly, easy-goi...",https://a0.muscache.com/pictures/41011975/0cdf...,320564,...,4.84,4.48,f,2,2,0,0,0.71,asheville,12
1,1,155305,https://www.airbnb.com/rooms/155305,20211215062309,2021-12-15,Cottage! BonPaul + Sharky's Hostel,<b>The space</b><br />Private cottage located ...,"We are within easy walk of pubs, breweries, mu...",https://a0.muscache.com/pictures/8880711/cf38d...,746673,...,4.93,4.53,t,7,1,2,4,2.75,asheville,12
2,2,156805,https://www.airbnb.com/rooms/156805,20211215062309,2021-12-15,"Private Room ""Ader"" at BPS Hostel",<b>The space</b><br />Private Rooms at Bon Pau...,"Easy walk to pubs, cafes, bakery, breweries, l...",https://a0.muscache.com/pictures/23447d55-fa7e...,746673,...,4.84,4.46,t,7,1,2,4,0.54,asheville,12
3,3,156926,https://www.airbnb.com/rooms/156926,20211215062309,2021-12-15,"Mixed Dorm ""Top Bunk #1"" at BPS Hostel",This is a top bunk in the mixed dorm room<br /...,NaN,https://a0.muscache.com/pictures/98f4e655-c4d6...,746673,...,4.78,4.78,t,7,1,2,4,2.36,asheville,12
4,4,160594,https://www.airbnb.com/rooms/160594,20211215062309,2021-12-15,Historic Grove Park,Come enjoy the beautiful Grove Park neighborho...,NaN,https://a0.muscache.com/pictures/92433837/d340...,769252,...,4.90,4.85,f,1,0,1,0,0.46,asheville,12


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 713564 entries, 0 to 713563
Data columns (total 76 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   Unnamed: 0                                    713564 non-null  int64  
 1   id                                            713564 non-null  int64  
 2   listing_url                                   713564 non-null  object 
 3   scrape_id                                     713564 non-null  int64  
 4   last_scraped                                  713564 non-null  object 
 5   name                                          713503 non-null  object 
 6   description                                   698474 non-null  object 
 7   neighborhood_overview                         460770 non-null  object 
 8   picture_url                                   713560 non-null  object 
 9   host_id                                       71

### Remove columns with too many Nan

In [5]:
df.isna().sum()[df.isna().sum() >= 0.5 * len(df)]

neighbourhood_group_cleansed    383183
bathrooms                       713564
calendar_updated                713564
dtype: int64

In [7]:
na_cols = list(df.isna().sum()[df.isna().sum() >= 0.5 * len(df)].index)
na_cols

['neighbourhood_group_cleansed', 'bathrooms', 'calendar_updated']

In [8]:
df_dropNA = df[df.columns.drop(na_cols)]

In [9]:
df_dropNA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 713564 entries, 0 to 713563
Data columns (total 73 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   Unnamed: 0                                    713564 non-null  int64  
 1   id                                            713564 non-null  int64  
 2   listing_url                                   713564 non-null  object 
 3   scrape_id                                     713564 non-null  int64  
 4   last_scraped                                  713564 non-null  object 
 5   name                                          713503 non-null  object 
 6   description                                   698474 non-null  object 
 7   neighborhood_overview                         460770 non-null  object 
 8   picture_url                                   713560 non-null  object 
 9   host_id                                       71

### Impute the rest nan with most frequent

In [55]:
df_cleaned = df_dropNA.fillna(df_dropNA.agg(lambda x: pd.Series.mode(x)[0], axis=0))

In [56]:
sum(df_cleaned.isna().sum())

0

In [57]:
df_cleaned['host_response_rate'] = df_cleaned['host_response_rate'].map(lambda x: x.rstrip('%'))
df_cleaned['host_acceptance_rate'] = df_cleaned['host_acceptance_rate'].map(lambda x: x.rstrip('%'))

In [58]:
df_cleaned['host_response_rate'] = df_cleaned['host_response_rate'].astype(int)
df_cleaned['host_acceptance_rate'] = df_cleaned['host_acceptance_rate'].astype(int)

### find variables that can be changed to numeric

- String: price
- Boolean: host_is_superhost, host_has_profile_pic, host_identity_verified, has_availability, instant_bookable


In [59]:
df_cleaned['price'] = df_cleaned.price.str.replace("$", "").str.replace(",", "").astype(float)

/var/folders/2t/5fzjt2x13h329_bxg11ttqzh0000gn/T/ipykernel_18124/2786629010.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_cleaned['price'] = df_cleaned.price.str.replace("$", "").str.replace(",", "").astype(float)


In [73]:
mapping = {'t': 1, 'f': 0}
df_cleaned = df_cleaned.replace({'host_is_superhost': mapping, 
                                 'host_has_profile_pic': mapping,
                                 'host_identity_verified': mapping, 
                                 'has_availability': mapping,
                                 'instant_bookable': mapping
                                })

In [75]:
np.unique(df_cleaned['host_is_superhost'])

array([0, 1])

### Use only numeric variables

In [76]:
num_col = []

for col in df_cleaned.columns.tolist():
    if ((df_cleaned[col].dtypes == 'int64') | (df_cleaned[col].dtypes == 'float64')):
        num_col.append(col)

In [77]:
num_col

['Unnamed: 0',
 'id',
 'scrape_id',
 'host_id',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_listings_count',
 'host_total_listings_count',
 'host_has_profile_pic',
 'host_identity_verified',
 'latitude',
 'longitude',
 'accommodates',
 'bedrooms',
 'beds',
 'price',
 'minimum_nights',
 'maximum_nights',
 'minimum_minimum_nights',
 'maximum_minimum_nights',
 'minimum_maximum_nights',
 'maximum_maximum_nights',
 'minimum_nights_avg_ntm',
 'maximum_nights_avg_ntm',
 'has_availability',
 'availability_30',
 'availability_60',
 'availability_90',
 'availability_365',
 'number_of_reviews',
 'number_of_reviews_ltm',
 'number_of_reviews_l30d',
 'review_scores_rating',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value',
 'instant_bookable',
 'calculated_host_listings_count',
 'calculated_host_listings_count_entire_homes',
 'calculated_host_listings_c

In [78]:
for col in ['host_id','Unnamed: 0', 'scrape_id', 'latitude', 'longitude', 'host_listings_count']:
    num_col.remove(col)

In [79]:
len(num_col)

41

In [80]:
dicAggFunc = {}

for key in num_col:
    dicAggFunc[key] = np.mean   

In [81]:
dicAggFunc['id'] = pd.Series.nunique

In [82]:
df_arranged = pd.DataFrame(df_cleaned.pivot_table(index = 'host_id', 
                                                  values = num_col,
                                                  aggfunc = dicAggFunc
                                                 ))
df_arranged.head()

,accommodates,availability_30,availability_365,availability_60,availability_90,bedrooms,beds,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,...,number_of_reviews_ltm,price,review_scores_accuracy,review_scores_checkin,review_scores_cleanliness,review_scores_communication,review_scores_location,review_scores_rating,review_scores_value,reviews_per_month
host_id,,,,,,,,,,,,,,,,,,,,,
23,2.000,1.333333,78.916667,3.666667,7.916667,1.0,1.000,3.0,0.0,3.0,...,0.083333,65.583333,5.741667,5.833333,5.366667,5.833333,5.77500,19.883333,5.57500,0.406667
470,3.000,0.000000,0.000000,0.000000,0.000000,1.0,2.000,1.0,0.0,1.0,...,0.000000,68.000000,9.000000,10.000000,10.000000,10.000000,10.00000,90.000000,9.00000,0.150000
521,6.000,0.000000,137.666667,0.000000,0.000000,2.0,3.000,1.0,1.0,0.0,...,0.000000,115.000000,5.000000,3.000000,5.000000,4.000000,5.00000,4.000000,4.00000,0.020000
615,3.000,6.125000,333.875000,31.250000,60.000000,1.0,1.625,2.0,2.0,0.0,...,21.125000,110.000000,6.185000,6.237500,6.197500,6.208750,6.16125,28.015000,5.98375,6.376250
666,2.125,2.875000,54.125000,9.000000,20.750000,1.5,1.500,2.0,2.0,0.0,...,19.500000,122.750000,6.201250,6.242500,6.208750,6.238750,6.19000,28.462500,6.16000,6.603750


In [83]:
df_arranged = df_arranged.rename(columns={"id": "listing_numbers"})

In [84]:
df_arranged.columns

Index(['accommodates', 'availability_30', 'availability_365',
       'availability_60', 'availability_90', 'bedrooms', 'beds',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'has_availability',
       'host_acceptance_rate', 'host_has_profile_pic',
       'host_identity_verified', 'host_is_superhost', 'host_response_rate',
       'host_total_listings_count', 'listing_numbers', 'instant_bookable',
       'maximum_maximum_nights', 'maximum_minimum_nights', 'maximum_nights',
       'maximum_nights_avg_ntm', 'minimum_maximum_nights',
       'minimum_minimum_nights', 'minimum_nights', 'minimum_nights_avg_ntm',
       'month', 'number_of_reviews', 'number_of_reviews_l30d',
       'number_of_reviews_ltm', 'price', 'review_scores_accuracy',
       'review_scores_checkin', 'review_scores_cleanliness',
       'review_scores_communication', '

In [85]:
df_arranged['annual_potential_revenue'] = df_arranged.listing_numbers * df_arranged.minimum_nights_avg_ntm * df_arranged.reviews_per_month * df_arranged.price * 12

In [86]:
df_arranged.head()

,accommodates,availability_30,availability_365,availability_60,availability_90,bedrooms,beds,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,...,price,review_scores_accuracy,review_scores_checkin,review_scores_cleanliness,review_scores_communication,review_scores_location,review_scores_rating,review_scores_value,reviews_per_month,annual_potential_revenue
host_id,,,,,,,,,,,,,,,,,,,,,
23,2.000,1.333333,78.916667,3.666667,7.916667,1.0,1.000,3.0,0.0,3.0,...,65.583333,5.741667,5.833333,5.366667,5.833333,5.77500,19.883333,5.57500,0.406667,1.600233e+03
470,3.000,0.000000,0.000000,0.000000,0.000000,1.0,2.000,1.0,0.0,1.0,...,68.000000,9.000000,10.000000,10.000000,10.000000,10.00000,90.000000,9.00000,0.150000,1.224000e+10
521,6.000,0.000000,137.666667,0.000000,0.000000,2.0,3.000,1.0,1.0,0.0,...,115.000000,5.000000,3.000000,5.000000,4.000000,5.00000,4.000000,4.00000,0.020000,8.280000e+02
615,3.000,6.125000,333.875000,31.250000,60.000000,1.0,1.625,2.0,2.0,0.0,...,110.000000,6.185000,6.237500,6.197500,6.208750,6.16125,28.015000,5.98375,6.376250,2.019996e+04
666,2.125,2.875000,54.125000,9.000000,20.750000,1.5,1.500,2.0,2.0,0.0,...,122.750000,6.201250,6.242500,6.208750,6.238750,6.19000,28.462500,6.16000,6.603750,4.537797e+05


In [87]:
df_arranged.annual_potential_revenue.describe()

count    1.358760e+05
mean     1.890995e+05
std      3.466138e+07
min      0.000000e+00
25%      3.723818e+03
50%      1.225970e+04
75%      3.780000e+04
max      1.224000e+10
Name: annual_potential_revenue, dtype: float64

In [120]:
df_arranged.describe()

,accommodates,availability_30,availability_365,availability_60,availability_90,bedrooms,beds,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,...,price,review_scores_accuracy,review_scores_checkin,review_scores_cleanliness,review_scores_communication,review_scores_location,review_scores_rating,review_scores_value,reviews_per_month,annual_potential_revenue
count,135876.000000,135876.000000,135876.000000,135876.000000,135876.000000,135876.000000,135876.000000,135876.000000,135876.000000,135876.000000,...,135876.000000,135876.000000,135876.000000,135876.000000,135876.000000,135876.000000,135876.000000,135876.000000,135876.000000,1.358760e+05
mean,3.830888,8.600388,139.035777,21.225733,36.022956,1.642514,2.066336,1.769764,1.251332,0.481816,...,216.723401,5.547004,5.593482,5.475337,5.591072,5.543824,17.452007,5.471278,1.811995,1.890995e+05
std,2.467422,9.307278,122.353064,19.718398,30.517268,0.981355,1.592021,5.122003,4.644347,1.933613,...,413.276985,1.182052,1.186259,1.185807,1.184060,1.176810,21.140310,1.162009,2.745246,3.466138e+07
min,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000e+00
25%,2.000000,0.000000,17.250000,0.500000,2.666667,1.000000,1.000000,1.000000,0.000000,0.000000,...,85.000000,4.962500,5.000000,4.920000,5.000000,4.950000,4.923333,4.880000,0.522500,3.723818e+03
50%,3.000000,5.500000,114.000000,17.250000,33.000000,1.000000,1.761905,1.000000,1.000000,0.000000,...,131.250000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,1.000000,1.225970e+04
75%,5.000000,14.000000,243.500000,36.000000,61.333333,2.000000,2.814815,1.000000,1.000000,1.000000,...,224.000000,6.190000,6.225000,6.140000,6.222500,6.169167,28.182500,6.100000,2.180000,3.780000e+04
max,16.000000,30.000000,365.000000,60.000000,90.000000,17.000000,132.000000,428.489736,428.489736,292.334322,...,25000.000000,10.000000,10.000000,10.000000,10.000000,10.000000,100.000000,10.000000,252.730000,1.224000e+10


In [116]:
len(np.unique(df.host_id))

135876

In [114]:
len(df_arranged)

135876

In [117]:
df_arranged.to_csv("./arranged_num.csv", index_label="host_id")

In [119]:
df_test = pd.read_csv("./arranged_num.csv")
len(df_test)

135876